## Wildfire Data Acquisition
This notebook contains the code that was used to process the GeoJson Data file. This code use pool processing and some of the snippets where a result of shared collaboration. I would like to provide Attribution to Raagul Nagendran. 

In [ ]:
%%capture
%pip install geojson pyproj
import geojson
import multiprocessing
import json

In [4]:
from compute_distance import process_fire_feature

In [5]:
geojson_file = open("USGS_Wildland_Fire_Combined_Dataset.json","r")
gj_data = geojson.load(geojson_file)
geojson_file.close()

In [7]:
for key in gj_data:
    print(f"{key} has {len(gj_data[key])} values")

displayFieldName has 0 values
fieldAliases has 30 values
geometryType has 19 values
spatialReference has 2 values
fields has 30 values
features has 135061 values


In [9]:
print("The wildfire feature from slot 0 of the loaded gj_data['features']")
gj_data["features"][0]

The wildfire feature from slot 0 of the loaded gj_data['features']


{'attributes': {'OBJECTID': 1,
  'USGS_Assigned_ID': 1,
  'Assigned_Fire_Type': 'Wildfire',
  'Fire_Year': 1860,
  'Fire_Polygon_Tier': 1,
  'Fire_Attribute_Tiers': '1 (1)',
  'GIS_Acres': 3940.20708940724,
  'GIS_Hectares': 1594.5452365353703,
  'Source_Datasets': 'Comb_National_NIFC_Interagency_Fire_Perimeter_History (1)',
  'Listed_Fire_Types': 'Wildfire (1)',
  'Listed_Fire_Names': 'Big Quilcene River (1)',
  'Listed_Fire_Codes': 'No code provided (1)',
  'Listed_Fire_IDs': '',
  'Listed_Fire_IRWIN_IDs': '',
  'Listed_Fire_Dates': 'Listed Other Fire Date(s): 2006-11-02 - NIFC DATE_CUR field (1)',
  'Listed_Fire_Causes': '',
  'Listed_Fire_Cause_Class': 'Undetermined (1)',
  'Listed_Rx_Reported_Acres': None,
  'Listed_Map_Digitize_Methods': 'Other (1)',
  'Listed_Notes': '',
  'Processing_Notes': '',
  'Wildfire_Notice': 'Wildfire mapping prior to 1984 was inconsistent, infrequent, and done without the aid of more modern fire mapping methods (GPS and satellite imagery). Areas burned

In [16]:
def check_attributes(gj_data):
    """Checks for missing attributes in each fire feature of the GeoJSON data.

    Args:
        gj_data: A GeoJSON data structure.

    Prints:
        A message indicating missing attributes for each fire feature with missing data.
    """

    required_attributes = ['Fire_Year', 'Listed_Fire_Names', 'GIS_Acres', 'Assigned_Fire_Type']

    for feature in gj_data['features']:
        missing_attributes = []
        for attr in required_attributes:
            if attr not in feature['attributes']:
                missing_attributes.append(attr)

        if missing_attributes:
            wf_year = feature['attributes'].get('Fire_Year', 'N/A')
            wf_name = feature['attributes'].get('Listed_Fire_Names', 'N/A').split(',')[0]
            print(f"Fire '{wf_name}' from {wf_year} is missing the following attributes: {missing_attributes}")

# Example usage:
check_attributes(gj_data)

## Calculate distance

In [17]:
place = {
    'city'   : 'Rialto',
    'latlon' : [34.106117, -117.372093]
}

In [18]:
fires_info = {}
fires_errors = {}

# Create a multiprocessing pool
pool = multiprocessing.Pool()

# Prepare input data for the pool
input_data = [(wf_feature, place, i) for i, wf_feature in enumerate(gj_data["features"])]

# Process the fire features in parallel
results = pool.starmap(process_fire_feature, input_data)

# Close the pool and wait for all processes to finish
pool.close()
pool.join()

# Process the results
for result in results:
    key, fire = result
    if isinstance(fire, dict):
        fires_info[key] = fire
    else:
        fires_errors[key] = fire

In [19]:
print(f"Distance information calculated for {len(fires_info)} fires in the US.")
first_fire = list(fires_info.keys())[0]
print(f"Information for {first_fire}:")
fires_info[first_fire]

Distance information calculated for 135033 fires in the US.
Information for 1860-0-Big Quilcene River (1):


{'year': 1860,
 'name': 'Big Quilcene River (1)',
 'size': 3940.20708940724,
 'type': 'Wildfire',
 'distance_shortest': [990.5529674481241,
  (47.81486350163385, -123.03396103506907)],
 'avg_distance': 991.8264977665281,
 'perimeter_start': (47.82096426863667, -123.03391282044802)}

In [20]:
print(f"We were unable to calculate the distance for {len(fires_errors)} fires due to various issues.")
list(fires_errors.keys())

We were unable to calculate the distance for 28 fires due to various issues.


['2007-109604-BARNES SOUTH UNDERBURN - BARNES SOUTH  UNDERBURN (1)',
 '2007-110223-BOYD JOHNSON (1)',
 '2008-110638-VIRGINIAS (3)',
 '2008-111430-GEORGE BRAND (1)',
 '2009-111896-VIRGINIAS (2)',
 '2009-112409-JAMIE JORDON (1)',
 '2009-112414-JIM TIMBERLAKE (1)',
 '2010-113410-No Fire Name Provided (1)',
 '2010-113664-WILLIAM GHOLSTON (1)',
 '2010-113737-JOHN HUTCHESON (1)',
 '2010-113765-JEAN DODSON (1)',
 "2010-113804-KEITH O'DANIEL (1)",
 '2010-114308-MICHAEL CROWE (1)',
 '2010-114321-JEANETTE WALKER (1)',
 '2012-115628-No Fire Name Provided (2)',
 '2012-115973-No Fire Name Provided (1)',
 '2012-116234-Little Bald Hills - Williams Ridge Rx (2)',
 '2013-117085-BALANCE UNDERBURN UNIT 2-1 (1)',
 '2015-119581-MYRTLE BB WUI FY 15 (1)',
 '2015-119616-RICHARDS',
 '2015-119750-DELTA BLOCK 1 (1)',
 '2015-119981-JIM BETHEL GFC (1)',
 '2015-120211-INTERDISCIPLINARY UNIT LBJ 41 - PATCH BURN UNIT A (HOPWELL CHURCH) (1)',
 '2016-120677-FULTON (1)',
 '2016-121009-No Fire Name Provided (1)',
 '2017-

In [21]:
list(fires_errors.values())

[KeyError(0),
 KeyError(0),
 KeyError(0),
 KeyError(0),
 KeyError(0),
 KeyError(0),
 KeyError(0),
 KeyError(0),
 KeyError(0),
 KeyError(0),
 KeyError(0),
 KeyError(0),
 KeyError(0),
 KeyError(0),
 KeyError(0),
 KeyError(0),
 KeyError(0),
 KeyError(0),
 KeyError(0),
 KeyError(0),
 KeyError(0),
 KeyError(0),
 KeyError(0),
 KeyError(0),
 KeyError(0),
 KeyError(0),
 KeyError(0),
 KeyError(0)]

In [22]:
# Format the keyerror values to string so that it can be stored in JSON
for key, value in fires_errors.items():
    fires_errors[key] = str(value)

In [24]:
# savig the data to json file for further use.
def write_to_json(filename, data):
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4)
        print(f"{filename} successfully created!")

write_to_json("fire_distance_info.json", fires_info)
write_to_json("fire_distance_errors.json", fires_errors)

fire_distance_info.json successfully created!
fire_distance_errors.json successfully created!
